## Preprocesamiento
1. Conseguir las imágenes, decir origen
2. Reescalarlas
3. Suavizado (explorar con y sin suavizado)
4. Transformar todas las imagenes a dataframe con informacion de cada pixel por columna

Ya tenemos realizado los primeros pasos. Ahora debemos cargar las imagenes para poder armar la base de datos.

In [64]:
import cv2
from skimage import data, color, io
from skimage.transform import rescale, resize, downscale_local_mean
import skimage
from skimage import io

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os

from sklearn.model_selection import train_test_split

import pandas as pd
%matplotlib inline


In [75]:
def suavizar(img):
    """
    Suaviza a la imagen aplicando kernel de convolucion
    """
    n = 2
    kernel = np.ones((n,n),np.float32)*(1./(n**2))
    img_tf = cv2.filter2D(img,-1,kernel)
    return img_tf

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

def load_data(path,w,h):
    """
    Crea la base de datos de las imagenes
    args:
        path: path de la carpeta con imagenes
        w: ancho de imagen
        h: alto de imagen
    returns:
        dataframe con registros de imagenes
    """
    names = os.listdir(path)
    width = w
    height = h
    data = np.zeros([len(names),width*height])
    for i in range(len(names)):
        image = io.imread(path+"/"+names[i], as_gray=True)
        image_resized = resize(image,(height , width),anti_aliasing=True)
        data[i,:] = np.asarray(image_resized).reshape(-1)
    data = pd.DataFrame(data,columns=[str(i) for i in range(width*height)])
    return data

In [76]:
path_con = "C:/Users/juank/Dropbox/Semestre_7/Machine_Learning/data_tapabocas/con_tapabocas"
path_sin = "C:/Users/juank/Dropbox/Semestre_7/Machine_Learning/data_tapabocas/sin_tapabocas"
width = 72
height = 128
df_con = load_data(path_con,width,height)
df_sin = load_data(path_sin,width,height)

In [60]:
print(df_con.shape)
print(df_sin.shape)
df = pd.DataFrame(np.concatenate([df_con,df_sin],axis=0), columns = df_con.columns)
df["mask_on"] = np.array([1]*df_con.shape[0] + [0]*df_sin.shape[0])
df = df.sample(frac=1)

(21, 9216)
(20, 9216)


In [61]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,mask_on
9,0.673712,0.668589,0.657381,0.655152,0.659804,0.657715,0.649664,0.643483,0.642445,0.641218,...,0.380772,0.378526,0.229850,0.288693,0.469107,0.512106,0.526431,0.555686,0.579604,1
16,0.479534,0.477747,0.486388,0.574758,0.589286,0.584315,0.364553,0.441822,0.400314,0.360930,...,0.279212,0.278056,0.271242,0.267067,0.261666,0.256782,0.256090,0.255030,0.259705,1
27,0.927410,0.932512,0.936917,0.943463,0.951398,0.958142,0.965493,0.969615,0.972305,0.976393,...,0.364676,0.380318,0.349866,0.335801,0.325940,0.316771,0.326206,0.346864,0.370612,0
10,0.506074,0.507949,0.512580,0.513993,0.515096,0.515203,0.516741,0.517311,0.521036,0.523645,...,0.427159,0.431209,0.438848,0.433876,0.412276,0.408612,0.408743,0.378064,0.369202,1
7,0.693578,0.719368,0.715441,0.713789,0.719102,0.731281,0.738810,0.744845,0.753127,0.765367,...,0.317282,0.332105,0.338403,0.334767,0.336747,0.340084,0.346526,0.351830,0.254755,1


----

## Implementación de redes neuronales
### Separación de datos de entrenamiento y de prueba

In [68]:
X = df.iloc[:,:-1]
y = df["mask_on"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [69]:
print(len(X_train))
print(len(X_test))

32
9


In [77]:
# Inicializando los parametros de la red neuronal

from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='sigmoid', input_shape=(width*height,)))
network.add(layers.Dense(10,activation='sigmoid'))
network.compile(optimizer = 'rmsprop',
                loss='categorical_crossentropy',
               metrics=['accuracy'])

In [81]:
network.weights

[<tf.Variable 'dense/kernel:0' shape=(9216, 512) dtype=float32, numpy=
 array([[-0.01596468, -0.00246531, -0.00083495, ..., -0.00050855,
         -0.01832999, -0.01281285],
        [ 0.00269997, -0.01289133,  0.01940166, ...,  0.02015766,
         -0.02469348,  0.00631034],
        [-0.00883722, -0.00743679,  0.0001919 , ..., -0.01376786,
         -0.0002823 , -0.02097998],
        ...,
        [ 0.00831966,  0.01636033,  0.01525248, ..., -0.00207865,
          0.01386422, -0.01163787],
        [ 0.01892413, -0.01608491,  0.01920578, ...,  0.01716822,
         -0.01302474,  0.0237868 ],
        [ 0.01982917,  0.02321663, -0.01809654, ..., -0.00918127,
         -0.00986598, -0.00973535]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(512,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [78]:
network.fit(X_train, y_train, epochs=7, batch_size = 128)

Epoch 1/7


ValueError: in user code:

    C:\Users\juank\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\juank\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\juank\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\juank\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\juank\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\juank\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 